In [ ]:
import sys, os

loc = os.environ.get('KAGGLE_KERNEL_RUN_TYPE','Localhost')
if loc == 'Batch' or loc == 'Interactive':
    !pip install --no-index --find-links=../input/coleridge-packages/spacy spacy

In [12]:
from pathlib import Path
import json, re

import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher
print(spacy.__version__)

In [8]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

In [9]:
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')

dataset_labels = set(train_df['dataset_label'].unique())
dataset_titles = set(train_df['dataset_title'].unique())
dataset_refs = list(dataset_labels.union(dataset_titles))

In [10]:
nlp = spacy.load("en_core_web_sm")
nlp.select_pipes(enable="")
matcher = PhraseMatcher(nlp.vocab)
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in dataset_refs]
matcher.add("TerminologyList", patterns)

In [22]:
publications = Path("../input/coleridgeinitiative-show-us-the-data/test/").iterdir()

submission_entries = []
for publication_path in publications:
    pub_matches = []
    pub_entry = {'Id': publication_path.stem}
    with open(publication_path, "r") as f:
        publication = json.load(f)
    for section in publication:
        doc = nlp(section['text'])
        for match_id, start, end in matcher(doc):
            pub_matches.append(clean_text(doc[start:end].text))
    pub_matches = set(pub_matches)
    pub_entry['PredictionString'] = '|'.join(pub_matches)
    submission_entries.append(pub_entry.copy())

In [25]:
pd.DataFrame(submission_entries).to_csv("submission.csv", index=None)